import 函式庫

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as img
import os
import statsmodels.api as sm
from statsmodels.formula.api import ols

連結Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

讀取.csv

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/A_stat_data_112.csv", encoding="UTF-8")
df

印出Label

In [ ]:
for i in df:
    print(i)

清洗資料

In [ ]:
df.drop(["mission_datetime_utc8", "completed_at_utc8"], axis=1, inplace=True)
df.dropna(subset=["mission_difficulty"], inplace=True)
df

千萬別用這個東東

In [ ]:
# def del_not_completed(row):
#     clear_output(wait=True)  # 清除上一個輸出
#     print(f"正在處理行索引: {row.name}")
#     if row["is_completed"] == 0:
#         df.drop(index=row.name, inplace=True)

# df.apply(del_not_completed, axis=1)

# print("處理完成！")

篩掉沒有完成的項

In [ ]:
df = df[df['is_completed'] == 1]
df.head(6)

In [ ]:
mission_list = df["mission_name"].unique()
mission_list = list(mission_list)
mission_list

In [ ]:
import tkinter as tk
from tkinter import ttk

root = tk.Tk()
root.title('PaGameO')
root.geometry('600x500')
root.configure(bg='gray57')

def show(*e):
    a.set(value.get())  # Label 變數改變成選單內容，使用 get() 取值

def search():
    # 取得選單的當前值
    selected_value = value.get()
    # 清空結果區域的內容
    result_text.delete(1.0, tk.END)
    # 高效篩選與選擇值相關的資料（假設篩選目標為 '描述' 欄位）
    filtered_df = df[df['mission_name'].str.contains(selected_value, na=False)]
    # 印出標題
    result_text.insert(tk.END, "用戶 ID\t\t\t完成時間\n")
    result_text.insert(tk.END, "-------------------------------------\n")
    # 印出篩選結果
    if filtered_df.empty:
        result_text.insert(tk.END, "無相關資料")
    else:
        for _, row in filtered_df.iterrows():
            result_text.insert(tk.END, f"{row['user_id']:<20}\t  {row['mission_continue_time']}\n")
            result_text.insert(tk.END, "-------------------------------------\n")

# Label 變數
a = tk.StringVar()
a.set('05/23 一起來看臺灣芒果【多元文化教育－異國美食】（初階）')

label = tk.Label(root, textvariable=a)  # 建立 Label，文字為變數
label.pack()

# 建立 Combobox
value = tk.StringVar()
value.set(mission_list[0])  # 設定預設選項

combo = ttk.Combobox(root, textvariable=value, values=mission_list, state="readonly", width=50)
combo.pack()

# 綁定事件，當選項改變時執行 show 函數
value.trace('w', show)

# 搜尋按鈕
search_button = tk.Button(root, text="搜尋", command=search)
search_button.pack(pady=10)

# 顯示 DataFrame 的文字區域
result_text = tk.Text(root, height=15, width=60)
result_text.pack()

root.mainloop()

主程式碼(操作)

In [ ]:
X=sm.add_constant(df['mission_difficulty'])
print(X)
y=df['first_correct_rate']
print(y)
model = sm.OLS(y, X).fit()
# Display the regression results
print(model.summary())

         const mission_difficulty
10         1.0                  易
11         1.0                  中
40         1.0                  難
64         1.0                  難
65         1.0                  難
...        ...                ...
1909904    1.0                  難
1909905    1.0                  易
1909909    1.0                  難
1909913    1.0                  易
1909917    1.0                  難

[428062 rows x 2 columns]
10         0.0
11         0.4
40         0.2
64         0.2
65         0.4
          ... 
1909904    0.4
1909905    0.2
1909909    0.0
1909913    0.2
1909917    0.2
Name: first_correct_rate, Length: 428062, dtype: float64


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [ ]:
# 將數據轉換為DataFrame
df_test = pd.DataFrame(df[['mission_difficulty', 'first_correct_rate']])
# 使用statsmodels 中的ols 方法進行單因子ANOVA 分析
model = ols('first_correct_rate ~ C(mission_difficulty)', data=df_test).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
# 輸出ANOVA 表
print(anova_table)
print(df_test)

                             sum_sq        df           F         PR(>F)
C(mission_difficulty)     63.564438       2.0  307.287866  4.392910e-134
Residual               44058.121014  425978.0         NaN            NaN
        mission_difficulty  first_correct_rate
10                       易                 0.0
11                       中                 0.4
40                       難                 0.2
64                       難                 0.2
65                       難                 0.4
...                    ...                 ...
1909904                  難                 0.4
1909905                  易                 0.2
1909909                  難                 0.0
1909913                  易                 0.2
1909917                  難                 0.2

[428062 rows x 2 columns]
